In [1]:
!pip install transformers datasets accelerate peft

  Using cached torch-2.7.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cufft_cu12-11.3.0.4-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.7.1.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 2.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━ 360.2/821.0 MB 2.2 MB/s eta 0:03:31ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device

   ━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━ 360.4/821.0 MB 2.2 MB/s eta 0:03:30

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import csv
import json
import re
import os

input_file = "/home/samrawit/Desktop/Amharic-E-commerce-Data-Extractor/data/raw/cleanedData.csv"
output_file = "/home/samrawit/Desktop/Amharic-E-commerce-Data-Extractor/data/processed/labeled_data.conll"
os.makedirs(os.path.dirname(output_file), exist_ok=True)

def is_amharic(token):
    return bool(re.search(r'[\u1200-\u137F]', token))

def is_english(token):
    return bool(re.match(r'[A-Za-z]', token))

def label_tokens(tokens):
    labels = ['O'] * len(tokens)

    # 1. PRODUCT → first 1–4 English tokens
    for i in range(min(5, len(tokens))):
        if is_english(tokens[i]) and tokens[i].lower() not in ['size', 'price']:
            if labels[i] == 'O':
                labels[i] = 'B-Product' if i == 0 or labels[i-1] == 'O' else 'I-Product'
        else:
            break

    # 2. PRICE → after 'Price' or 'በ', before 'birr' or 'ብር' or 
    for i, token in enumerate(tokens):
        if token.lower() == "price" or token == "በ" or token == "ዋጋ":
            if i+1 < len(tokens) and re.match(r'^\d{2,6}$', tokens[i+1]):
                labels[i] = 'B-PRICE'
                labels[i+1] = 'I-PRICE'
                if i+2 < len(tokens) and tokens[i+2].lower() in ['birr', 'ብር']:
                    labels[i+2] = 'I-PRICE'

    # 3. LOCATION → after 'አድራሻ'
    for i, token in enumerate(tokens):
        if token == "አድራሻ":
            labels[i] = 'B-LOC'
            for j in range(1, 5):
                if i+j < len(tokens) and is_amharic(tokens[i+j]):
                    labels[i+j] = 'I-LOC'
                else:
                    break

    # 4. CONTACT_INFO → 09xxxxxxxx or @username
    for i, token in enumerate(tokens):
        if re.match(r'^09\d{8}$', token) or token.startswith("@"):
            labels[i] = 'B-CONTACT_INFO'

    return labels

# 🚀 Read CSV and write CoNLL
with open(input_file, "r", encoding="utf-8") as infile, \
     open(output_file, "w", encoding="utf-8") as outfile:

    reader = csv.DictReader(infile)

    for row in reader:
        try:
            tokens = json.loads(row["tokens"])
        except Exception:
            continue

        labels = label_tokens(tokens)

        for token, label in zip(tokens, labels):
            outfile.write(f"{token} {label}\n")
        outfile.write("\n")

print("✅ Rule-based labeling complete. Saved to:", output_file)